top 可以看 cpu 狀態
sudo powermetrics --samplers cpu_power,gpu_power --show-initial-usage 可以看到 GPU 狀態

In [ ]:
%pip install experiment-impact-tracker


In [2]:
import psutil
import time

p = psutil.Process()
start_cpu = p.cpu_times().user + p.cpu_times().system
start_wall = time.time()

print("starting")
# ... your code here ...

end_cpu = p.cpu_times().user + p.cpu_times().system
end_wall = time.time()

cpu_usage_factor = (end_cpu - start_cpu) / (end_wall - start_wall)
print(f"CPU usage factor: {cpu_usage_factor:.2f}")


starting
CPU usage factor: 1.10


In [ ]:
import torch
print(torch.cuda.is_available())
# false means there are no GPUs available
print(torch.backends.mps.is_available()) 

False
True


In [3]:
import sys
import tempfile

import torch
from experiment_impact_tracker.compute_tracker import ImpactTracker


class Experiment:
    def __init__(self):
        device = torch.device("cpu")
        # device = torch.device('cuda') # Uncomment this to run on GPU

        # N is batch size; D_in is input dimension;
        # H is hidden dimension; D_out is output dimension.
        N, D_in, H, D_out = 1024, 10000, 1000, 100

        # Create random input and output data
        self.x = torch.randn(N, D_in, device=device)
        self.y = torch.randn(N, D_out, device=device)

        # Randomly initialize weights
        self.w1 = torch.randn(D_in, H, device=device)
        self.w2 = torch.randn(H, D_out, device=device)
        self.learning_rate = 1e-6

    def train(self):
        # Forward pass: compute predicted y
        h = self.x.mm(self.w1)
        h_relu = h.clamp(min=0)
        y_pred = h_relu.mm(self.w2)

        # Compute and print loss; loss is a scalar, and is stored in a PyTorch Tensor
        # of shape (); we can get its value as a Python number with loss.item().
        loss = (y_pred - self.y).pow(2).sum()

        # Backprop to compute gradients of w1 and w2 with respect to loss
        grad_y_pred = 2.0 * (y_pred - self.y)
        grad_w2 = h_relu.t().mm(grad_y_pred)
        grad_h_relu = grad_y_pred.mm(self.w2.t())
        grad_h = grad_h_relu.clone()
        grad_h[h < 0] = 0
        grad_w1 = self.x.t().mm(grad_h)

        # Update weights using gradient descent
        self.w1 -= self.learning_rate * grad_w1
        self.w2 -= self.learning_rate * grad_w2


def my_experiment() -> None:
    tmp_dir = tempfile.mkdtemp()
    # Init tracker with log path
    tracker = ImpactTracker(tmp_dir)
    # Start tracker in a separate process
    tracker.launch_impact_monitor()

    exp = Experiment()

    for t in range(100):
        if t % 10 == 9:
            print(f"Pass: {t}")
            # Optional. Adding this will ensure that your experiment stops if impact tracker throws an exception and exit.
            tracker.get_latest_info_and_check_for_errors()
        exp.train()

    print(f"Please find your experiment logs in: {tmp_dir}")


if __name__ == "__main__":
    my_experiment()

ImportError: cannot import name 'json_normalize' from 'pandas.io.json' (/Users/lulu/miniconda3/lib/python3.12/site-packages/pandas/io/json/__init__.py)